In [23]:
#

import pandas as pd
import numpy as np
import sys, random, argparse, os, re, time



In [3]:
tweets = pd.read_csv('tweets_10-17-2020.csv')
tweets.head(10)
# len(tweets['text'][7])

,id,text,isRetweet,device,favorites,retweets,date
0,115846232714391552,My interview which recently aired on CNBC's Sq...,f,TweetDeck,7,18,2011-09-19 05:54:28
1,98482327390396416,http://bit.ly/pwgGsQ My interview yesterday on...,f,TweetDeck,12,13,2011-08-02 07:56:31
2,98474279192113153,"The more you learn about the debt deal, the wo...",f,TweetDeck,24,175,2011-08-02 07:24:32
3,98466170625196033,Your questions about my desk answered in today...,f,Twitter Web Client,8,22,2011-08-02 06:52:19
4,98454970654916608,Republicans and Democrats have both created ou...,f,TweetDeck,40,262,2011-08-02 06:07:48
5,1243755076214231040,"Such Fake reporting by the @nytimes, @washingt...",f,Twitter for iPhone,141942,34544,2020-03-28 04:21:32
6,1270411061376815108,My decision to appoint @usairforce General Cha...,f,Twitter for iPhone,157337,37398,2020-06-09 17:42:54
7,1256589289602908161,RT @SexCounseling: @realDonaldTrump As far as ...,t,Twitter for iPhone,0,5037,2020-05-02 14:20:07
8,1255283103691661313,"RT @GLFOP: Thank you, President @realDonaldTru...",t,Twitter for iPhone,0,8827,2020-04-28 23:49:48
9,1304767518775611394,....It has now been determined that the Muelle...,f,Twitter for iPhone,99088,26890,2020-09-12 13:03:12


In [57]:
class Markov(object):
    def __init__(self, _list, _make_new, multiple = True, lower = False):
        '''Multiple: True if phrases are separated in a list, 
                     False if it's a unique text (won't know where to start the next phrase)'''
        self._list     = _list
        self._tokenlen = _tokenlen
        self.multiple  = multiple
        self.lower     = lower
        self._make_new()
    def _make_new(self):
        if self.multiple:
            self.chain = {'initials_':{}}
            # analyze each text individually and store the first values in the 'initials_' dictionary
            for phrase in self._list:
                for _tuple in self._get_tuples(phrase):
                    #print(_tuple)
                    text_ = ' '.join(_tuple[0]).lower() if self.lower else ' '.join(_tuple[0])
                    next_ = _tuple[1][-1].lower()       if self.lower else _tuple[1][-1]
                    index = _tuple[2]
                    if index == 0:
                        self.chain['initials_'][ text_ ] = self.chain['initials_'].get( text_, 0 ) + 1
                    if text_ not in self.chain:
                        entry = self.chain[ text_ ] = {}
                    else:
                        entry = self.chain[ text_ ]
                    entry[ next_ ] = entry.get( next_, 0) + 1
        #self.chain = {'initials_':{}} if self.multiple else {}
    def _get_tuples(self, phrase):
        phrase = phrase.split()
        for i in range(0, len(phrase) - self._tokenlen):
            yield (phrase[ i: i+self._tokenlen], phrase[ 1+i : i+1+self._tokenlen ], i)
    def update(self, _list=None, _tokenlen=None, what='replace'):
        '''Allows to change the chain parameters without creating a new object'''
        changed = False
        if _tokenlen is not None:
            #print('Changed chain length')
            self._tokenlen = _tokenlen
            changed = True
        if _list is not None:
            #print('Changed chain elements')
            if what == 'replace':
                self._list = _list
            elif what == 'add':
                self._list = self.list_ + _list
            changed = True
        if changed:
            self._make_new()
    def generate_text(self, _len = 10):
        if self.multiple:
            text = self._generate_next_letter( 'initials_' ).split( ' ' )
        else:
            text = np.random.choice( self.chain.keys() ).split( ' ' )
        _len -= self._tokenlen
        while _len > 0:
            newkey = text[ -self._tokenlen : -1 ]
            text.append( self._generate_next_letter( 'initials_' ) )
            _len -= 1
        return ' '.join(text)
            
        print(text)
    def _generate_next_letter(self, token):
        items = self.chain[token]
        rnd = random.randint( 0, sum(items.values()) - 1 ) 
        #print(rnd)
        for item in items:
            rnd -= items[item]
            if rnd < 0:
                #print(item)
                return item
        
        
        
    


In [58]:
_tokenlen = 3
phrases = tweets.text.values.tolist()
# print(phrases[:5])
tt = Markov( phrases, _tokenlen, multiple = True, lower = False )


In [61]:
# # tt.chain
# print("*************")
# print(tt.generate_text(10))
# print("*************")
# tt.update( _tokenlen = 5 )
# # tt.chain['initials_']
# print("*************")
# print(tt.generate_text(10))
# print("*************")
# tt.update( _tokenlen = 2 )
# print("*************")
# print(tt.generate_text(10))
# print("*************")
for i in range(10, 1, - 1):
    print("************************************")
    tt.update(_tokenlen=i)
    print(i)
    print(tt.generate_text(random.randint(i + 5, i + 20)))
    print("************************************")

************************************
10
.@WSJ and dopey Karl Rove made a mistake and purposely RT @SarahHuckabee: The Atlantic story on @realDonaldTrump is total BS. "Just reported by CNN that ""the Trump halo effect"" caused RT @stillgray: This guy needs a therapist more than he The dishonest media didn't mention that Bernie Sanders was very """@ThePrimeHost: @realDonaldTrump LOVE the new signage. Keep doing what you've """@vetteman88: @realDonaldTrump @DMRegister @JenniferJJacobs we sure could use someone who RT @AnthemRespect: If you’re an Independent or Moderate Democrat... ...never RT @BrandonStraka: I started #WalkAway w no money. No political Sorry Chief Justice John Roberts, but you do indeed have """@ArthurKade: Fun chatting w/ Derek Hough, Donald Trump, Zac Posen, Canada will now sell its oil to China because @BarackObama “Stock had the Best First Half since 1997.” @WallStreetJour Our RT @WhiteHouse: This Sunday, millions of Christians will celebrate Easter. RT @markpmeredith